In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [2]:
from springstone.data import get_data, get_missing_dates, create_train_test, create_df_for_prophet, download_model
from springstone.utils import bollinger_bands, moving_average, prophet_preprocessing, prophet_non_business_days
from springstone.utils import basic_recommendation, temp_data_predict, next_business_day
from springstone.trainer import Trainer
from springstone.predict import predict_from_model
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_forecast_component_plotly
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import joblib
from termcolor import colored
from datetime import date, timedelta
import holidays
from tensorflow.keras.models import load_model

2022-03-12 15:47:20.150642: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-12 15:47:20.150690: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2022-03-12 15:47:21,953 loading file /home/blueoverseas/.flair/models/sentiment-en-mix-distillbert_4.pt


In [3]:
ticker = 'AAPL'

In [4]:
hist = get_data(ticker, end='2022-03-12')

[*********************100%***********************]  1 of 1 completed


In [5]:
hist

,Open,High,Low,Close,Volume
Date,,,,,
2012-02-21,18.102858,18.387501,18.004286,18.387501,605595200
2012-02-22,18.324286,18.410357,18.181070,18.322857,483302400
2012-02-23,18.395714,18.493929,18.196428,18.442499,568027600
2012-02-24,18.559643,18.674999,18.522858,18.657499,415072000
2012-02-27,18.618214,18.875000,18.438572,18.777143,547582000
...,...,...,...,...,...
2022-03-07,163.360001,165.020004,159.039993,159.300003,96418800
2022-03-08,158.820007,162.880005,155.800003,157.440002,131148300
2022-03-09,161.479996,163.410004,159.410004,162.949997,91454900


In [6]:
df_nbd = prophet_non_business_days(hist)

In [7]:
df_train, df_test = create_train_test(hist)

In [9]:
# Using the Trainer class
trainer_prophet = Trainer(model="prophet",X=df_train, y=None, non_business_days=df_nbd)
df_test_prophet = prophet_preprocessing(df_test, 'Close')
trainer_prophet.run()
trainer_prophet.save_model_locally(ticker,'prophet')

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -43.7746
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       7268.53     0.0130693       1031.53           1           1      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       7321.36    0.00159364       362.424           1           1      243   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     278       7330.59    0.00123625       363.072   5.172e-06       0.001      383  LS failed, Hessian reset 
     299       7333.22     0.0145204       843.769      0.3809           1      407   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     319       7335.82   0.000116299       176.341   1.536e-06       0.001      468  LS failed, Hessian reset 
     383       7337.62   0.000115938       138.322   2.581e-06       0.001      585  LS failed, Hessian rese

In [11]:
# load RNN model
rnn_model = load_model(f'../{ticker}')

2022-03-12 15:50:05.904727: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-12 15:50:05.904773: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-12 15:50:05.904792: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (MacBookPro15): /proc/driver/nvidia/version does not exist
2022-03-12 15:50:05.905008: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-12 15:50:07.917536: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 1

In [12]:
#preprocess the input for RNN prediction
preprocessor = joblib.load(f'../preprocessor_{ticker}.joblib')
df_test_preprocessed = preprocessor.transform(df_test)
df_test_preprocessed = df_test_preprocessed[-14:,:]
df_test_preprocessed = np.expand_dims(df_test_preprocessed, axis=0)
df_test_preprocessed.shape

(1, 14, 26)

In [13]:
# RNN next day closing price prediction
scaled_close_prediction = rnn_model.predict(df_test_preprocessed)
fin_scaller_output = np.zeros((1,7))
fin_scaller_output[0,1] = scaled_close_prediction
unscaled_close_prediction = preprocessor.named_steps['transform'].named_transformers_['fin_scaller'].inverse_transform(fin_scaller_output)[0,1]
unscaled_close_prediction

90.86665007101715

In [14]:
df_test.loc[date.today()] = np.full((1,5), np.nan).tolist()[0]
df_test

,Open,High,Low,Close,Volume
Date,,,,,
2020-03-11 00:00:00,69.347504,70.305000,67.964996,68.857498,255598800.0
2020-03-12 00:00:00,63.985001,67.500000,62.000000,62.057499,418474000.0
2020-03-13 00:00:00,66.222504,69.980003,63.237499,69.492500,370732000.0
2020-03-16 00:00:00,60.487499,64.769997,60.000000,60.552502,322423600.0
2020-03-17 00:00:00,61.877499,64.402496,59.599998,63.215000,324056000.0
...,...,...,...,...,...
2022-03-08 00:00:00,158.820007,162.880005,155.800003,157.440002,131148300.0
2022-03-09 00:00:00,161.479996,163.410004,159.410004,162.949997,91454900.0
2022-03-10 00:00:00,160.199997,160.389999,155.979996,158.520004,105342000.0


In [ ]:
df_test_prophet

In [ ]:
y_test_pred = predict_from_model(ticker, 'prophet', pd.concat([df_test_prophet[['ds']], pd.DataFrame({'ds': [date.today()]})]))
y_test_pred

In [ ]:
np.full((1,5),np.nan).shape

In [ ]:
df_test

In [ ]:
pd.concat([df_test[['Close']], pd.DataFrame()])

In [ ]:
df_test

In [ ]:
pd.concat([pd.concat([df_test[['Close']].loc[date.today()] = np.nan]), pd.DataFrame({'Close_pred_prophet': y_test_pred.values}, index=df_test.index)], axis=1)

In [ ]:
n_days_prediction=1
future = pd.DataFrame({'ds': [date.today()]})
future

In [ ]:
forecast = predict_from_model(ticker, 'prophet', future)
forecast

In [ ]:
mae_prophet = mean_absolute_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
mape_prophet = mean_absolute_percentage_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
print(f'MAE: {mae_prophet}')
pbrint(f'MAPE: {mape_prophet}')

In [ ]:
def predict_from_model(ticker, model_type, X):
    """Predicts from a saved model for a ticker and a model_type
       Input:
            ticker: ticker used by the model
            model_type: type of model used such as prohet or RNN
            X: data
       Ouptut: return a dataframe with predicted values"""
    model = download_model(ticker, model_type)
    if model_type == "prophet":
        y_pred = model['prophet_model'].prophet.predict(X)['yhat']
    else:
        y_pred = model.predict(X)
        
    return y_pred

In [ ]:
cd notebooks

In [ ]:
download_model(ricker, 'prophet')